###### 2020-10-07 화요일

# 페이지를 이동하며 크롤링하기 : 동적크롤링

### - 목차
##### 1. 이미지 스크랩
##### 2. 네이버 블로그 스크랩
##### 3. 동적스크랩 : 네이버 쇼핑 (selenium 사용 x)

## 1. 이미지 스크랩

> - 네이버 이미지에서 `코로나`를 검색하고 스크랩을 진행해봅시다.
> - 이 과정은 url을 두 파트로 나누어 하나는 변하지 않는 부분 (`base_url`), 하나는 변하는 부분(`key_word`)로 나누어 진행할 거에요. 이렇게 하는 이유는 나중에 동적으로 페이지를 이동시키면서 동적인 스크랩을 진행하기 위해서, 연습하는거라고 생각하면 될거 같습니다 ㅎㅎ 

In [7]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import HTTPError
from urllib.error import URLError
from urllib.parse import quote_plus

#### (1) url 두 부분으로 나누기
> - base_url은 url에서 변하지않는 부분 / key_word는 url에서 변하는 부분입니다

In [4]:
base_url = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query="
key_word = input('검색어 입력 :')
imgCnt = int(input('스크랩 할 이미지 개수 :'))

try:
    url = base_url + quote_plus(key_word)  # url 주소에 한글이 들어가면안됨, quote_plus는 한글을 변형시켜주는 함수
    html = urlopen(url)
except HTTPError as he:
    print("http error")
except URLError as ue:
    print("url error")
else :
    soup = BeautifulSoup(html.read(), "html.parser", from_encoding='UTF-8')

검색어 입력 :코로나
스크랩 할 이미지 개수 :50


### (2) 이미지 파일 추출 / 저장하기

> - `img` 태그를 찾아 이미지를 추출해 봅시다

In [33]:
img = soup.findAll("img", {"class" : "_img"})

###### 이미지를 추출하고 저장하는 과정입니다.

In [34]:
cnt = 1
for i in img :
    imgUrl = i['data-source']
    with urlopen(imgUrl) as file:
        with open('./images/' + str(cnt) + '.jpg', 'wb') as imgFile:
            temp = file.read()
            imgFile.write(temp)
    cnt += 1
    if cnt > imgCnt :
        break
print('Image download success')

Image download success


##### 이 파일과 동일한 디렉토리에 있는 `images`폴더에 이미지가 저장되어 있음을 확인해보세요!

## 2. 네이버 블로그 스크랩
> - 이 파트 역시 url을 두 부분으로 나누어 진행할 거에요~ 동적스크랩 연습이라고 보시면 됩니다!
> - 네이버 블로그에서 `파이썬`을 검색하고 첫 페이지를 스크랩할 거에요
> - 마찬가지로 `base_url`은 url의 변하지 않는 부분, `key_word` url은 변하는 부분이에요

In [44]:
base_url = "https://search.naver.com/search.naver?where=post&sm=tab_jum&query="
key_word = input('검색어 입력 :')

try:
    url = base_url + quote_plus(key_word)
    html = urlopen(url)
except HTTPError as he:
    print("http error")
except URLError as ue:
    print("url error")
else :
    soup = BeautifulSoup(html.read(), "html.parser", from_encoding='UTF-8')

검색어 입력 :파이썬


### (1) 네이버 블로그 스크랩하기
> - 블로그의 제목과 링크를 스크랩하는 과정이에요

In [45]:
title = soup.findAll("a", {"class" : "sh_blog_title _sp_each_url _sp_each_title"})

# 블로그 제목
title_list = [element['title'] for element in title]
print(title_list)

# 블로그 링크
link_list = [element['href'] for element in title]
print(link_list)

['파이썬 숫자 출력하기_하루 10분 혼공', '[Python] 파이썬이 인기있는 이유', '파이썬 웹 프로그래밍 Django로 입문', '[P056] 파이썬의 정형화된 출력형식(Formatted Output of Python)', '알고리즘 트레이딩 - 파이썬을 활용한 금융 API 준비하는 방법은?', '[아두이노 QnA] 파이썬으로 아두이노 제어하기_2', '파이썬인강 프로그래밍 막막하셨다면', '[책 구매 / 리뷰] 파이썬 알고리즘 인터뷰', '파이썬 인강 ☞ Python 기초 개념부터 배우기 초보자 특별 강좌', '파이썬독학 충분히 쉽다는 책']
['https://blog.naver.com/nasu0210?Redirect=Log&logNo=222105195536', 'https://blog.naver.com/dktmrorl?Redirect=Log&logNo=222104270383', 'https://blog.naver.com/urmyver?Redirect=Log&logNo=222079389364', 'https://blog.naver.com/choi_s_h?Redirect=Log&logNo=222103125184', 'https://blog.naver.com/ridesafe?Redirect=Log&logNo=222069600885', 'https://blog.naver.com/dokkosam?Redirect=Log&logNo=222091055778', 'https://blog.naver.com/skyktc?Redirect=Log&logNo=222070357069', 'https://blog.naver.com/dsz08082?Redirect=Log&logNo=222097605931', 'https://blog.naver.com/beta700?Redirect=Log&logNo=222091992623', 'https://blog.naver.com/gogwangjin?Redirect=Log&logNo=222107477542']


### (2) 스크랩한 데이터 csv파일로 저장하기
> - 스크랩한 데이터를 DataFrame으로 저장하고 `blog_scrap.csv`파일로 저장합니다.

In [54]:
import pandas as pd

data = {"title" : title_list,
       "link" : link_list}

df = pd.DataFrame(data)
df.to_csv("blog_scrap.csv", mode='w', encoding='utf-8', index=False) # 판다스는 기본적으로 index열을 포함하여 csv파일로 저장한다
                                                                      # 그래서 index를 제거하고 저장

### (3) 저장한 csv파일 불러보기

In [58]:
# 저장한 csv 파일을 불러옵시다

read_df = pd.read_csv("./blog_scrap.csv", encoding="utf-8")
read_df

,title,link
0,파이썬 숫자 출력하기_하루 10분 혼공,https://blog.naver.com/nasu0210?Redirect=Log&l...
1,[Python] 파이썬이 인기있는 이유,https://blog.naver.com/dktmrorl?Redirect=Log&l...
2,파이썬 웹 프로그래밍 Django로 입문,https://blog.naver.com/urmyver?Redirect=Log&lo...
3,[P056] 파이썬의 정형화된 출력형식(Formatted Output of Python),https://blog.naver.com/choi_s_h?Redirect=Log&l...
4,알고리즘 트레이딩 - 파이썬을 활용한 금융 API 준비하는 방법은?,https://blog.naver.com/ridesafe?Redirect=Log&l...
5,[아두이노 QnA] 파이썬으로 아두이노 제어하기_2,https://blog.naver.com/dokkosam?Redirect=Log&l...
6,파이썬인강 프로그래밍 막막하셨다면,https://blog.naver.com/skyktc?Redirect=Log&log...
7,[책 구매 / 리뷰] 파이썬 알고리즘 인터뷰,https://blog.naver.com/dsz08082?Redirect=Log&l...
8,파이썬 인강 ☞ Python 기초 개념부터 배우기 초보자 특별 강좌,https://blog.naver.com/beta700?Redirect=Log&lo...
9,파이썬독학 충분히 쉽다는 책,https://blog.naver.com/gogwangjin?Redirect=Log...


### 3. 동적스크랩 : 네이버 쇼핑 (selenium 사용 x)
> - 지금까지 url을 두 부분으로 나누어 스크랩을 진행해보았습니다. 이번에는 두 부분으로 나누고 (`base_url`과 `key_word`) `key_word`부분을 바꿈으로써 페이지를 넘겨 동적 스크랩을 진행해 봅시다.
> - 예제는 네이버 쇼핑입니다

In [77]:
from time import sleep
from random import randint
import pandas as pd
from IPython.core.display import clear_output
from time import time
from bs4 import BeautifulSoup
from requests import get


### (1) 시간계산 함수 실습해보기
> - 동적스크랩(=크롤링)의 경우, 네이버와 같은 홈페이지에서 시간 딜레이를 적용하지 않고 시행하게되면, 홈페이지에서 크롤링 행위를 홈페이지 공격 행위로 간주하여 IP를 차단할 수 있습니다. 그래서 1~2초 정도의 딜레이를 주는것이 좋아요. 그래서 한번 연습해보는 단계라고 보시면 됩니다!

In [4]:
# loop 사이에 잘 진행되고 있는지를 보기위한 indicator sentence

start_time = time()
# start_time  # 밀리언 세컨드로 나오기 때문에 의미는 없다


requests = 0
for _ in range(5):
    requests += 1
    sleep(randint(1, 3))
    current_time = time()
    elapsed_time = current_time - start_time
    print("request : {}, frequency : {}" .format(requests, requests/elapsed_time))
    clear_output(wait=True)



request : 5, frequency : 0.45411056992310866


### (2) 네이버 쇼핑 동적스크랩 실습
> - 이전 시간에 배웠던 `tqdm` 패키지를 적용해서 동적 스크랩을 진행해 볼까요?

In [8]:
base_url = "https://search.naver.com/search.naver?where=post&sm=tab_jum&query="
key_word = input('검색어 입력 :')

try:
    url = base_url + quote_plus(key_word)
    html = urlopen(url)
except HTTPError as he:
    print("http error")
except URLError as ue:
    print("url error")
else :
    soup = BeautifulSoup(html.read(), "html.parser", from_encoding='UTF-8')

검색어 입력 :파이썬


In [9]:
title = soup.findAll("a", {"class" : "sh_blog_title _sp_each_url _sp_each_title"})

In [55]:
pages = [str(num) for num in range(1, 11)]
pages

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [80]:
from tqdm import tqdm_notebook

requests = 0

title_list = []
price_list = []
link_list = []

start_time = time()

for page in tqdm_notebook(pages):
    requests += 1
    
    # 네이버 쇼핑 URL
    html = urlopen("https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery=%EC%BD%94%ED%8A%B8&pagingIndex=" + page + "&pagingSize=40&productSet=total&query=%EC%BD%94%ED%8A%B8&sort=rel&timestamp=&viewType=list")
    
    sleep(1) # 동적크롤링을 빠르게 시행하는 경우 홈페이지에 대한 공격으로 간주되어 내 IP가 블락 당할수있다. 그러므로 1~2초 여유를 두는것
             # time 패키지에 있는 sleep 함수 
    soup = BeautifulSoup(html.read(), "html.parser")
    
    temp_list = soup.findAll("li", {"class" : "basicList_item__2XT81"})
    
    for element in tqdm_notebook(temp_list) :
        title_list.append(element.find("a", {"class" : "basicList_link__1MaTN"}).text)
        price_list.append(element.find("span", {"class" : "price_num__2WUXn"}).text)
        link_list.append(element.find("a", {"class" : "basicList_link__1MaTN"})['href'])
        
    current_time = time()
    elapsed_time = current_time - start_time
    print('처리페이지 : ', page)
    print('걸린시간 : ', elapsed_time)


C:\Users\lan41\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


C:\Users\lan41\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



처리페이지 :  1
걸린시간 :  1.5657811164855957



처리페이지 :  2
걸린시간 :  3.2293341159820557



처리페이지 :  3
걸린시간 :  4.80013632774353


...


처리페이지 :  6
걸린시간 :  9.727962732315063



처리페이지 :  7
걸린시간 :  11.390520095825195



처리페이지 :  8
걸린시간 :  12.916439294815063



처리페이지 :  9
걸린시간 :  14.388506650924683



처리페이지 :  10
걸린시간 :  16.112894296646118



In [82]:
## 한페이지에서 5개의 상품 정보밖에 가져오지 못하고 있다
# 왜그런진 모르겟어 ㅠㅠ


print(len(title_list))
print(len(price_list))
print(len(link_list))

50
50
50


In [83]:
## 데이터프레임 만들고 csv파일 저장하기

data = {"title" : title_list,
       "price" : price_list,
       "link" : link_list}

df = pd.DataFrame(data)

df.to_csv("shopping_data.csv", encoding='UTF-8', index=False)